<a href="https://colab.research.google.com/github/mjgpinheiro/AI-Strategies-StockMarket/blob/master/Options_IB_MLv1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from ibapi.client import EClient
from ibapi.wrapper import EWrapper
from ibapi.contract import Contract
from ibapi.common import BarData
from ibapi.order import Order
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier

class IBapi(EWrapper, EClient):
    def __init__(self):
        EClient.__init__(self, self)
        self.historical_data = []

    def historicalData(self, reqId: int, bar: BarData):
        self.historical_data.append({
            'date': bar.date,
            'open': bar.open,
            'high': bar.high,
            'low': bar.low,
            'close': bar.close,
            'volume': bar.volume
        })

    def run_algorithm(self, contract, clf):
        # Get current market data
        self.reqMktData(1, contract, "", False, False, [])
        while len(self.historical_data) == 0:
            pass

        # Prepare data for classification
        data = pd.DataFrame(self.historical_data)
        data['price'] = data['close']
        data['volatility'] = data['high'] - data['low']
        data['action'] = 'SELL' if clf.predict(data[['price', 'volume', 'volatility']]) == 0 else 'BUY'

        # Place order based on predicted action
        if data['action'] == 'BUY':
            # Place a buy order
            order.action = "BUY"
            order.orderType = "LMT"
            order.totalQuantity = 1
            order.lmtPrice = 2.0
            self.placeOrder(self.nextOrderId, contract, order)
        elif data['action'] == 'SELL':
            # Place a sell order
            order.action = "SELL"
            order.orderType = "LMT"
            order.totalQuantity = 1
            order.lmtPrice = 2.0
            self.placeOrder(self.nextOrderId, contract, order)

    def optimize_strategy(self):
        # Load historical market data
        data = pd.read_csv('market_data.csv')

        # Define features and target
        X = data[['price', 'volume', 'volatility']]
        y = data['action']

        # Split data into training and testing sets
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

        # Train a decision tree classifier
        clf = DecisionTreeClassifier()
        clf.fit(X_train, y_train)

        return clf

app = IBapi()
app.connect("127.0.0.1", 7497, 0)

# Define contract for AAPL
contract = Contract()
contract.symbol = "AAPL"
contract.secType = "STK"
contract.exchange = "SMART"
contract.currency = "USD"

# Define order
order = Order()

# Optimize trading strategy
clf = app.optimize_strategy()

# Run algorithm
app.run_algorithm(contract, clf)

# Disconnect from API
app.disconnect()
